In [130]:
import pandas as pd
import numpy as np

In [131]:
vdf = pd.read_csv('fruitvegprices-2017_2022.csv')

In [132]:
vdf

,category,item,variety,date,price,unit
0,fruit,apples,bramleys_seedling,2022-03-11,2.05,kg
1,fruit,apples,coxs_orange_group,2022-03-11,1.22,kg
2,fruit,apples,egremont_russet,2022-03-11,1.14,kg
3,fruit,apples,braeburn,2022-03-11,1.05,kg
4,fruit,apples,gala,2022-03-11,1.03,kg
...,...,...,...,...,...,...
9642,cut_flowers,alstromeria,indoor,2017-11-03,0.27,stem
9643,cut_flowers,chrysanthemum,indoor_spray,2017-11-03,0.22,stem
9644,cut_flowers,lillies,oriental,2017-11-03,0.70,stem
9645,cut_flowers,narcissus,indoor,2017-11-03,0.06,stem


## Ejercicios

### Ejercicio uno: realice un analisis de valores nulos

En caso de existir valores nulos elimine esos registros

In [93]:
porcentaje_nulos = ((vdf.isnull().sum() / len(vdf))*100).sort_values(ascending = False)

In [94]:
porcentaje_nulos

category    0.0
item        0.0
variety     0.0
date        0.0
price       0.0
unit        0.0
dtype: float64

Nuestro análisis muestra que no existen valores nulos en nuestro dataset

### Ejercicio dos: Por categoria, cuantos distintos items hay y cuantos distintos variety hay?

El resultado debe de tener el siguiente formato de presentacion:

In [26]:
pd.DataFrame({
    "category" : ['cut_flowers','fruit']
    , "item_count" : [9,10]
    , "variety_Count" : [10,21]
})

,category,item_count,variety_Count
0,cut_flowers,9,10
1,fruit,10,21


Como nota en un mismo dataframe mostramos todo lo solicitado, usted debe de mostrar todas las categorias

In [95]:
vdf.columns

Index(['category', 'item', 'variety', 'date', 'price', 'unit'], dtype='object')

In [96]:
ej2 = vdf.loc[ : ,["category","item","variety"]]

In [97]:
ej2 = ej2.groupby(['category'],as_index=False).agg({'item' : ['count'], 'variety' : ['count']})

In [98]:
ej2.columns = ej2.columns.droplevel(1)

In [99]:
ej2.columns = ['category', 'item_count', 'variety_count']

In [100]:
ej2

,category,item_count,variety_count
0,cut_flowers,342,342
1,fruit,1992,1992
2,pot_plants,49,49
3,vegetable,7264,7264


### Ejercicio tres: Para la combinacion item, variety muestre el ultimo valor registrado por año.

Ejemplo de como debe de mostrar su respuesta

In [27]:
pd.DataFrame({
    "item" : ['Manzana']
    , "variety" : ['Gala']
    , "year" : [2022]
    , "last_price" :  [3.2]
})

,item,variety,year,last_price
0,Manzana,Gala,2022,3.2


In [394]:
vdf['date'] = pd.to_datetime(vdf['date'])

In [395]:
vdf['year'] = vdf['date'].dt.year

In [396]:
ej3 = vdf.loc[ : ,["item","variety","year","price"]]

In [397]:
ejj3 = ej3.sort_values(["item","variety","year"] , ascending = False )
ejj3["rank"] = ejj3.groupby(["item","variety","year"])["price"].rank( method = 'first' , ascending = False )
ejj3 = ejj3.loc[ ejj3["rank"] == 1.0 , : ].reset_index(drop=True)

In [398]:
ejj3.columns = ['item', 'variety', 'year', 'last_price', 'rank']

In [399]:
ejj3.drop(['rank'], axis=1)

,item,variety,year,last_price
0,watercress,pillow_packs,2019,10.16
1,watercress,pillow_packs,2018,12.68
2,watercress,pillow_packs,2017,11.44
3,turnip,turnip,2022,1.87
4,turnip,turnip,2021,1.83
...,...,...,...,...
392,apples,braeburn,2019,1.11
393,apples,braeburn,2018,1.05
394,apples,braeburn,2017,0.72
395,alstromeria,indoor,2018,0.34


### Ejercicio cuatro: Segun el ejercicio tres, que tanto aumento o disminuyo el precio de cada item-variety 

Ejemplo de como se debe de mostrar su respuesta

In [28]:
pd.DataFrame({
    "item" : ['Manzana','Manzana','Manzana']
    , "variety" : ['Gala','Gala','Gala']
    , "year" : [2020,2021,2022]
    , "last_price" :  [2.7,2.5,3.2]
    , "delta" : [0, (2.5-2.7), (3.2-2.5)]
})

,item,variety,year,last_price,delta
0,Manzana,Gala,2020,2.7,0.0
1,Manzana,Gala,2021,2.5,-0.2
2,Manzana,Gala,2022,3.2,0.7


Segun nuestro ejemplo dummy para 2021 estuvo 0.2 centavos mas barato y en 2022 estuvo 0.7 centavos caro

In [447]:
ej4 = ejj3.loc[ : ,["item","variety","year","last_price"]]

In [448]:
ej4['PreciosPrevios'] = ej4.groupby(["item"], as_index=False)["last_price"].shift(-1)

In [449]:
ej4 = ej4.sort_values(["item","variety","year"])

In [450]:
ej4['delta'] = ej4["last_price"] - ej4["PreciosPrevios"]

In [453]:
ej4 = ej4.drop(['PreciosPrevios'] ,axis = 1)

In [454]:
ej4.fillna(0)

,item,variety,year,last_price,delta
396,alstromeria,indoor,2017,0.29,0.00
395,alstromeria,indoor,2018,0.34,0.05
394,apples,braeburn,2017,0.72,0.00
393,apples,braeburn,2018,1.05,0.33
392,apples,braeburn,2019,1.11,0.06
...,...,...,...,...,...
4,turnip,turnip,2021,1.83,-0.17
3,turnip,turnip,2022,1.87,0.04
2,watercress,pillow_packs,2017,11.44,0.00
1,watercress,pillow_packs,2018,12.68,1.24


### Ejercicio cinco: Cual es precio promedio por item, variety

Ejemplo de como se debe de mostrar su respuesta

In [29]:
pd.DataFrame({
    "item" : ['Manzana']
    , "variety" : ['Gala']
    , "year-month" : ['2021-01']   
    , "avg-price" : [0.5]
})

,item,variety,year-month,avg-price
0,Manzana,Gala,2021-01,0.5


In [169]:
vdf['month'] = vdf['date'].dt.month

In [203]:
ej5 = vdf.loc[ : ,["item","variety","price","year","month"]]

In [204]:
ej5['year']= ej5['year'].apply(str)

In [205]:
ej5['month']= ej5['month'].apply(str)

In [206]:
ej5["year-month"] = ej5["year"] + "-" + ej5["month"]

In [207]:
ej5['year-moth'] = pd.to_datetime(ej5['year-month'])

In [208]:
ej5 = ej5.groupby(['item','variety','year-month'],as_index=False).agg({'price': ['mean']})

In [247]:
ej5.columns = ej5.columns.droplevel(1)

In [249]:
ej5.columns = ['item', 'variety', 'year-month', 'avg_price']

In [250]:
ej5

,item,variety,year-month,avg_price
0,alstromeria,indoor,2017-11,0.2700
1,alstromeria,indoor,2017-12,0.2800
2,alstromeria,indoor,2018-1,0.3100
3,alstromeria,indoor,2018-10,0.2350
4,alstromeria,indoor,2018-11,0.2580
...,...,...,...,...
2580,watercress,pillow_packs,2019-4,9.4750
2581,watercress,pillow_packs,2019-5,9.5000
2582,watercress,pillow_packs,2019-6,9.4900
2583,watercress,pillow_packs,2019-7,9.4975


### Ejercicio seis: muestre por category,item,variety que tanto % ha aumentado su precio

Ejemplo: si para las manzanas gala el primer precio que se registro fue 2.00 y el ultimo precio que se registro es 4.00 este aumento un 100%

In [36]:
pd.DataFrame({
    "category": ["fruta","fruta"]
    ,"item" : ['Manzana','Manzana']
    , "variety" : ['Gala','egremont_russet']
    , "percentage-change-between-first-and-last-price" : [200,150]       
})

,category,item,variety,percentage-change-between-first-and-last-price
0,fruta,Manzana,Gala,200
1,fruta,Manzana,egremont_russet,150


In [346]:
ej6 = vdf.loc[ : ,["category","item","variety","price"]]

In [347]:
lastv = ej6.sort_values(["category","item","variety"] , ascending = False )
lastv["rank"] = lastv.groupby(["category","item","variety"])["price"].rank( method = 'first' , ascending = False )
lastv = lastv.loc[ lastv["rank"] == 1.0 , : ].reset_index(drop=True)

Traemos el último valor para cada uno de los registros

In [348]:
firstv = ej6.sort_values(["category","item","variety"] , ascending = True )
firstv["rank"] = firstv.groupby(["category","item","variety"])["price"].rank( method = 'first' , ascending = True )
firstv = firstv.loc[ firstv["rank"] == 1.0 , : ].sort_values(["category","item","variety"] , ascending = False ).reset_index(drop=True)

Traemos el primer valor para cada uno de los registros

In [349]:
lastv['first_price'] = firstv['price']

In [350]:
lastv['percentage-change-between-first-and-last-price'] = lastv['price'] - lastv['first_price']

In [352]:
lastv['percentage-change-between-first-and-last-price'] = round((lastv['percentage-change-between-first-and-last-price'] / lastv['first_price'])*100,2)

Ejecutamos la fórmula para saber el porcentajde de aumento

In [353]:
porcentaje = lastv.loc[ : ,["category","item","variety","percentage-change-between-first-and-last-price"]]

In [354]:
porcentaje

,category,item,variety,percentage-change-between-first-and-last-price
0,vegetable,watercress,pillow_packs,93.59
1,vegetable,turnip,turnip,281.82
2,vegetable,tomatoes,vine,526.76
3,vegetable,tomatoes,round,271.11
4,vegetable,tomatoes,plum,718.18
...,...,...,...,...
80,cut_flowers,narcissus,indoor,125.00
81,cut_flowers,lillies,oriental,98.00
82,cut_flowers,gladioli,all_varieties,230.00
83,cut_flowers,chrysanthemum,indoor_spray,47.06


### Ejercicio siete: Segun el ejercicio seis por category cual es el item,variety con mayor aumento de crecimiento 

Segun el ejemplo dummy que se utilizo en el ejercicio seis para la category fruta se mostraria la manzana gala unicamente

In [355]:
ej7 = porcentaje.loc[ : ,["category","item","variety","percentage-change-between-first-and-last-price"]]

In [357]:
ej7 = ej7.groupby(["category","item","variety"],as_index=False).agg({'percentage-change-between-first-and-last-price' : ['max']})

In [363]:
ej7.columns = ej7.columns.droplevel(1)

In [370]:
by_percentage = ej7.sort_values('percentage-change-between-first-and-last-price',ascending=False)

In [373]:
by_percentage.columns = ['category', 'item', 'variety',
       'calor_por_item_cat_var_con_mayor_aumento_de_crecimiento']

In [374]:
by_percentage

,category,item,variety,calor_por_item_cat_var_con_mayor_aumento_de_crecimiento
30,fruit,strawberries,strawberries,974.00
29,fruit,raspberries,raspberries,831.22
79,vegetable,tomatoes,cherry,825.00
80,vegetable,tomatoes,plum,718.18
50,vegetable,capsicum,yelloworange,700.00
...,...,...,...,...
32,pot_plants,geranium,13_cm,60.00
0,cut_flowers,alstromeria,indoor,47.83
1,cut_flowers,chrysanthemum,indoor_spray,47.06
8,cut_flowers,sweet_williams,sweet_williams,14.29
